## Part 1: Real-time face detection

In [1]:
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.vision.face import FaceClient

import cv2
from IPython.display import Image
from matplotlib import pyplot as plt
from time import time

In [2]:
attributes_list = ['age', 'emotion', 'gender']
fps = 1
fps = fps * 1000

subscription_key = "3c4c8a7bbfa7427786b5644453e3377e"
endpoint = "https://visionfaceprateek.cognitiveservices.azure.com/"
cv_face_client = FaceClient(
    endpoint, CognitiveServicesCredentials(subscription_key))
print("Connected to resource", cv_face_client)

Connected to resource <azure.cognitiveservices.vision.face._face_client.FaceClient object at 0x000001C4FFD48040>


In [3]:
def process_frame(frame2api):
    cv2.imwrite('tmp_image.jpg', frame2api)
    tmp_img = open("tmp_image.jpg", 'rb')
    detected_faces = cv_face_client.face.detect_with_stream(
        tmp_img, return_face_attributes=attributes_list)
    tmp_img.close()
    return detected_faces

In [4]:
font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 1
color = (0, 0, 255)
thickness = 2


def process_result(frame2api, detected_faces):
    if len(detected_faces) > 0:
        print("Number of faces detected:", len(detected_faces))
        for faces_itr in range(len(detected_faces)):
            org = (detected_faces[faces_itr].face_rectangle.left,
                   detected_faces[faces_itr].face_rectangle.top)
            gender = detected_faces[faces_itr].face_attributes.gender.value
            age = detected_faces[faces_itr].face_attributes.age
            emotions_dict = detected_faces[faces_itr].face_attributes.emotion.as_dict(
            )
            emotion_name = max(
                zip(emotions_dict.values(), emotions_dict.keys()))[1]

            face_rect = [detected_faces[faces_itr].face_rectangle.left,
                         detected_faces[faces_itr].face_rectangle.top,
                         detected_faces[faces_itr].face_rectangle.width,
                         detected_faces[faces_itr].face_rectangle.height]

            frame2api = cv2.rectangle(frame2api, face_rect, (0, 0, 255), 5)
            frame2api = cv2.putText(frame2api, str(
                gender), org, font, fontScale, color, thickness, cv2.LINE_AA)
            frame2api = cv2.putText(frame2api, str(
                age), (org[0]+100, org[1]), font, fontScale, color, thickness, cv2.LINE_AA)
            frame2api = cv2.putText(frame2api, str(
                emotion_name), (org[0] + 200, org[1]), font, fontScale, color, thickness, cv2.LINE_AA)
    else:
        print("No Face Detected")
    return len(detected_faces)

In [5]:
vid = cv2.VideoCapture(0)
start_time = time() * 1000
init_frame = True

while (True):
    _, frame = vid.read()
    if init_frame:
        frame2api = frame.copy()
        detected_faces = process_frame(frame2api)
        process_result(frame2api, detected_faces)
        init_frame = False

    end_time = time() * 1000
    time_diff = end_time - start_time
    if (int(time_diff) >= fps):
        frame2api = frame.copy()
        detected_faces = process_frame(frame2api)
        process_result(frame2api, detected_faces)
        start_time = time() * 1000

    frame = cv2.putText(frame, "Real Time", (10,10), font,
                        fontScale, color, thickness, cv2.LINE_AA)
    frame2api = cv2.putText(frame2api, "Processed", (50,50),
                            font, fontScale, color, thickness, cv2.LINE_AA)
    frame_merged = cv2.hconcat([frame, frame2api])
    cv2.imshow("Display", frame_merged)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
vid.release()
cv2.destroyAllWindows()


Number of faces detected: 1
Number of faces detected: 1
Number of faces detected: 1
Number of faces detected: 1
Number of faces detected: 1
Number of faces detected: 1
Number of faces detected: 1
Number of faces detected: 1


## Part 2: Custom Vision

In [6]:
!pip install azure-cognitiveservices-vision-customvision

In [153]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials
import os, time, uuid

In [154]:
endpoint = "https://customvisionprateek.cognitiveservices.azure.com/"
training_key = "ff801d0322034b64b8091684cf72db55"
prediction_key = "e6c1b0b739da430fa32d61e59ed8ef21"
prediction_resource_id = "/subscriptions/d5d0e7cd-5900-4c80-820c-b24c2a8416d0/resourceGroups/PRATEEK-SINGH/providers/Microsoft.CognitiveServices/accounts/CustomVisionPrateek-Prediction"

In [155]:
credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(endpoint, credentials)
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(endpoint, prediction_credentials)

In [156]:
publish_iteration_name = "classifyDogCat-v3"

credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(endpoint, credentials)

# Create a new project
print ("Creating project...")
project_name = uuid.uuid4()
project = trainer.create_project(project_name)

Creating project...


In [157]:
dog_tag = trainer.create_tag(project.id, "Dog")
cat_tag = trainer.create_tag(project.id, "Cat")

#### Train using the dwonloaded dataset

In [159]:
base_image_location = os.path.dirname("C:\\Users\\singh\\Documents\\MastersAppliedAIDeakin\\"
                                      "SIT788_Engineering_AI_Solutions\\Assignments\\SIT788_5_2_Data\\dataset\\training_set\\")

print("Adding images...")

image_list = []
step = 32
itr = 32
#for itr in range(0, 100, step):
for image_num in range(itr, itr+step):
    file_name = "dog.{}.jpg".format(image_num+1)
    with open(os.path.join (base_image_location, "dogs", file_name), "rb") as image_contents:
        image_list.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), tag_ids=[dog_tag.id]))

for image_num in range(itr, itr+step):
    file_name = "cat.{}.jpg".format(image_num+1)
    with open(os.path.join (base_image_location, "cats", file_name), "rb") as image_contents:
        image_list.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), tag_ids=[cat_tag.id]))

upload_result = trainer.create_images_from_files(project.id, ImageFileCreateBatch(images=image_list))
if not upload_result.is_batch_successful:
    print("Image batch upload failed.")
    for image in upload_result.images:
        print("Image status: ", image.status)
    exit(-1)
else:
    print("Adding images successful")



Adding images...
Adding images successful


In [160]:
iteration = trainer.train_project(project.id)
while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project.id, iteration.id)
    print ("Training status: " + iteration.status)
    time.sleep(10)
print ("Training Done", iteration.status)
trainer.publish_iteration(project.id, iteration.id, publish_iteration_name, prediction_resource_id)
print ("Done!")

Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training sta

Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training sta

Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training sta

Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training sta

In [161]:
prediction_endpoint = "https://customvisionprateek-prediction.cognitiveservices.azure.com/"
test_image_location = os.path.dirname("C:\\Users\\singh\\Documents\\MastersAppliedAIDeakin\\"
                                      "SIT788_Engineering_AI_Solutions\\Assignments\\SIT788_5_2_Data\\dataset\\test_set\\")
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(prediction_endpoint, prediction_credentials)

with open(os.path.join (test_image_location, "dogs\\dog.4003.jpg"), "rb") as image_contents:
    results = predictor.classify_image(
        project.id, publish_iteration_name, image_contents.read())

    # Display the results.
    for prediction in results.predictions:
        print("\t" + prediction.tag_name +
              ": {0:.2f}%".format(prediction.probability * 100))

	Dog: 99.36%
	Cat: 0.67%


In [172]:
import glob
all_dogs_images = glob.glob(test_image_location + "/dogs/*")
all_cats_images = glob.glob(test_image_location + "/cats/*")

In [186]:
from tqdm import tqdm
TP = 0
FN = 0
TN = 0
FP = 0


for image_dog in tqdm(all_dogs_images):
    with open(os.path.join (test_image_location, image_dog), "rb") as image_contents:
        results = predictor.classify_image(
        project.id, publish_iteration_name, image_contents.read())
        
        for prediction in results.predictions:
            if prediction.probability > 0.5:
                prediction_name = prediction.tag_name
        if (prediction_name == "Dog"):
            TP += 1
        else:
            FN += 1
            
for image_cat in tqdm(all_cats_images):
    with open(os.path.join (test_image_location, image_cat), "rb") as image_contents:
        results = predictor.classify_image(
        project.id, publish_iteration_name, image_contents.read())
        
        for prediction in results.predictions:
            if prediction.probability > 0.5:
                prediction_name = prediction.tag_name
        if (prediction_name == "Cat"):
            TN += 1
        else:
            FP += 1


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [07:06<00:00,  2.34it/s]


In [178]:
Recall = TP / (TP + FN)
Precision = TP / (TP + FP)
F_Measure = (2 * Precision * Recall) / (Precision + Recall)
Accuracy = (TP + TN)/(TP + TN + FP + FN)

In [185]:
total = TP + TN + FP + FN
print ("Total classified samples: ", total)
print ("Evaluation metrics: Recall: ", Recall, "Precision: ", Precision, 
       "F-Measure: ", F_Measure, "Accuracy: ", Accuracy)

Total classified samples:  2000
Evaluation metrics: Recall:  0.981 Precision:  0.9969512195121951 F-Measure:  0.9889112903225806 Accuracy:  0.989


Total classified samples 2000
